In [26]:
import numpy as np
import matplotlib.pyplot as plt
import math

# Pre assigning Variables
* X = Strike Price
* volatility = Annual Volatility
* maturity = time to maturity
* riskfree = annual interest rate riskfree
* timestep = timestep (for binomial trees) 

In [27]:
X1 = 120 # Option with Strike price 120p
X2 = 96 # Option with Strike price 96p

In [28]:
s0 = 100 # Current Stock Price S0 = 100p
volatility = 0.2 # Annual volitility

In [29]:
maturity = 0.5 # time to maturity equal to 6 months (1/2 of a year).
riskfree = 0.05  # annual interest rate (with continuous compounding) of 5% riskless
timestep = round(182.5)

# Black Scholes Model
* The model is used to find the current value of a call option whose ultimate value depends on the price of the stock at the expiration date. Because the stock price keeps changing, the value of this call option will change too.

In [30]:
def norm_cdf(x):
    """
    Calculates the cumulative distribution function of the standard normal distribution.

    Parameters:
    x (float): the value to evaluate the CDF at

    Returns:
    float: the value of the CDF at x
    """
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

In [31]:
def bs(S, K, T, r, sigma, option_type):
    """
    Calculates the price of a European call or put option using the Black-Scholes formula.

    Parameters:
    S (float): current price of the underlying asset
    K (float): strike price of the option
    T (float): time to expiration of the option, expressed in years
    r (float): risk-free interest rate, expressed as a decimal
    sigma (float): annualized volatility of the underlying asset, expressed as a decimal
    option_type (str): 'call' or 'put'

    Returns:
    float: the price of the option according to the Black-Scholes formula
    """

    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)
    elif option_type == 'put':
        return K * math.exp(-r * T) * norm_cdf(-d2) - S * norm_cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'.")



In [32]:
x1putprice = bs(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, option_type='put')
x1callprice = bs(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, option_type='call')
x2putprice = bs(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, option_type='put')
x2callprice = bs(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, option_type='call')
print("X1 Put price:",x1putprice,"X1 Call price:",x1callprice)
print("X2 Put price:",x2putprice,"X2 Call price:",x2callprice)

X1 Put price: 18.05980466595578 X1 Call price: 1.0226152225558902
X2 Put price: 2.8519851786376655 X2 Call price: 9.222233623917731


# Binomial tree Method
* The binomial option pricing model is a risk-free method for estimating the value of path-dependent alternatives. With this model, investors can determine how likely they are to buy or sell at a given price in the future.

In [33]:
def trees(S, K, T, r, sigma, n, option_type, exercise_type):
    """
    Calculates the price of a European or American call or put option using the binomial tree method.

    Parameters:
    S (float): current price of the underlying asset
    K (float): strike price of the option
    T (float): time to expiration of the option, expressed in years
    r (float): risk-free interest rate, expressed as a decimal
    sigma (float): annualized volatility of the underlying asset, expressed as a decimal
    n (int): number of time steps in the binomial tree
    option_type (str): 'call' or 'put'
    exercise_type (str): 'European' or 'American'

    Returns:
    float: the price of the option according to the binomial tree method
    """
    
    delta_t = T / n
    u = math.exp(sigma * math.sqrt(delta_t))
    d = 1 / u
    p = (math.exp(r * delta_t) - d) / (u - d)
   
   
    

    # Compute the stock prices at all nodes in the tree
    stock_prices = []
    for i in range(n + 1):
        stock_prices.append(S * u**(n - i) * d**i)

    # Compute the option values at the final nodes in the tree
    option_values = []
    for price in stock_prices:
        if option_type == 'call':
            option_values.append(max(0, price - K))
        elif option_type == 'put':
            option_values.append(max(0, K - price))

    # Backward induction to compute the option values at earlier nodes
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_values[j] = math.exp(-r * delta_t) * (p * option_values[j] + (1 - p) * option_values[j + 1])
            if exercise_type == 'American':
                stock_prices[j] /= u
                early_exercise = 0
                if option_type == 'call':
                    early_exercise = max(0, stock_prices[j] - K)
                elif option_type == 'put':
                    early_exercise = max(0, K - stock_prices[j])
                option_values[j] = max(option_values[j], early_exercise)
                
                

  

    return option_values[0]

In [34]:
x1puteuro = trees(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='put', exercise_type='European')
x1calleuro = trees(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='call', exercise_type='European')
x1putamerican = trees(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='put', exercise_type='American')
x1callamerican = trees(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='call', exercise_type='American')
x2puteuro = trees(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='put', exercise_type='European')
x2calleuro = trees(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='call', exercise_type='European')
x2putamerican = trees(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='put', exercise_type='American')
x2callamerican = trees(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='call', exercise_type='American')
print("X1 Euro Put price:",x1puteuro,"X1 Euro Call price:",x1calleuro)
print("X1 American Put price:",x1putamerican,"X1 American Call price:",x1callamerican)
print("X2 Euro Put price:",x2puteuro,"X2 Euro Call price:",x2calleuro)
print("X2 American Put price:",x2putamerican,"X2 American Call price:",x2callamerican)


X1 Euro Put price: 18.059758635803597 X1 Euro Call price: 1.0225691924034679
X1 American Put price: 19.999999999999915 X1 American Call price: 1.0225691924034679
X2 Euro Put price: 2.847842715215464 X2 Euro Call price: 9.218091160495094
X2 American Put price: 2.98219776299529 X2 American Call price: 9.218091160495094


# Monte Carlo Method
* Monte Carlo is used for option pricing where numerous random paths for the price of an underlying asset are generated, each having an associated payoff. These payoffs are then discounted back to the present and averaged to get the option price.

In [35]:
def montecarlo(S, K, T, r, sigma, n, option_type, num_simulations=100000):
    """
    Calculates the price of a European call or put option using the Monte Carlo method.

    Parameters:
    S (float): current price of the underlying asset
    K (float): strike price of the option
    T (float): time to expiration of the option, expressed in years
    r (float): risk-free interest rate, expressed as a decimal
    sigma (float): annualized volatility of the underlying asset, expressed as a decimal
    n (int): number of time steps in the simulation
    option_type (str): 'call' or 'put'
    num_simulations (int): number of simulations to run

    Returns:
    float: the price of the option according to the Monte Carlo method
    """

    delta_t = T / n
    S_values = np.zeros(num_simulations)
    for i in range(num_simulations):
        # Generate a random path for the underlying asset price
        price = S
        for j in range(n):
            price *= np.exp((r - 0.5 * sigma**2) * delta_t + sigma * np.random.normal() * np.sqrt(delta_t))
        S_values[i] = price

    # Calculate the discounted payoff for each simulation
    if option_type == 'call':
        payoffs = np.maximum(S_values - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - S_values, 0)

    # Calculate the option price as the mean of the discounted payoffs
    option_price = np.exp(-r * T) * np.mean(payoffs)

    return option_price

In [36]:
x1putmonte =  montecarlo(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='put', num_simulations=100000)
x1callmonte = montecarlo(S=s0, K=X1, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='call', num_simulations=100000)
x2putmonte =  montecarlo(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='put', num_simulations=100000)
x2callmonte = montecarlo(S=s0, K=X2, T=maturity, r=riskfree, sigma=volatility, n=timestep, option_type='call', num_simulations=100000)
print("X1 Put price:",x1putmonte,"X1 Call price:",x1callmonte)
print("X2 Put price:",x2putmonte,"X2 Call price:",x2callmonte)

X1 Put price: 18.042037030029395 X1 Call price: 1.0117819571556323
X2 Put price: 2.843940025070798 X2 Call price: 9.17444300964768
